In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import warnings

In [7]:
# ============================================================
# SETUP & DATA LOADING
# ============================================================

calendar_df = pd.read_csv('../data/calendar.csv.gz', compression='gzip').sample(frac=0.1, random_state=42)
reviews = pd.read_csv('../data/reviews.csv.gz', compression='gzip').sample(frac=0.1, random_state=42)
listings = pd.read_csv('../data/listings.csv.gz', compression='gzip').sample(frac=0.1, random_state=42)

In [ ]:
# CALENDAR DATASET
# ============================================================
# 1: EXPLORATORY DATA ANALYSIS (EDA)
# ============================================================

In [ ]:
# Display dataset preview and missing value analysis
display(calendar_df.head(20))
print("MISSING VALUES OVERVIEW")
print("=" * 50)

missing_count = calendar_df.isnull().sum()
missing_percent = (calendar_df.isnull().sum() / len(calendar_df)) * 100
missing_summary = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Percent': missing_percent
})
print(missing_summary)

In [ ]:
# Drop price columns (100% missing, will merge from listings later)
calendar_df = calendar_df.drop(['price', 'adjusted_price'], axis=1)

In [ ]:
# # CALENDAR DATASET
# ============================================================
# 2: DATA CLEANING & PREPROCESSING
# ============================================================

In [ ]:
# Convert date to datetime and extract temporal features
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
calendar_df['year'] = calendar_df['date'].dt.year
calendar_df['month'] = calendar_df['date'].dt.month
calendar_df['day_of_week'] = calendar_df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
calendar_df['week_of_year'] = calendar_df['date'].dt.isocalendar().week
calendar_df['is_weekend'] = calendar_df['day_of_week'].isin([5, 6]).astype(bool)

# Convert 'available' to boolean
calendar_df['available'] = calendar_df['available'].map({'t': True, 'f': False})

# Set appropriate data types
calendar_df['listing_id'] = calendar_df['listing_id'].astype('int64')
calendar_df['minimum_nights'] = calendar_df['minimum_nights'].astype('int16')
calendar_df['maximum_nights'] = calendar_df['maximum_nights'].astype('int16')

In [ ]:
# Verify data integrity and display cleaned calendar dataset
print("Calendar Dataset Info:")
print(f"Shape: {calendar_df.shape}")
print(f"Any missing values: {calendar_df.isnull().sum().sum()}")
print("\nData types:")
print(calendar_df.dtypes)
print("\nFirst few rows:")
calendar_df.head()

In [ ]:
# Check for duplicate rows
print("Duplicate Analysis:")
print("=" * 50)
print(f"Total rows: {len(calendar_df)}")
print(f"Duplicate rows (all columns): {calendar_df.duplicated().sum()}")
print(f"Duplicate rows (listing_id, date): {calendar_df.duplicated(subset=['listing_id', 'date']).sum()}")

if calendar_df.duplicated(subset=['listing_id', 'date']).sum() > 0:
    print("\nRemoving duplicates on (listing_id, date)...")
    calendar_df = calendar_df.drop_duplicates(subset=['listing_id', 'date'], keep='first')
    print(f"New shape after removing duplicates: {calendar_df.shape}")
else:
    print("\nNo duplicates found on (listing_id, date) - dataset is clean!")

#No duplicates found

In [ ]:
# Check for extreme values and outliers
print("Extreme Values Analysis:")
print("=" * 50)

# Numeric columns to analyze
numeric_cols = ['minimum_nights', 'maximum_nights', 'month', 'day_of_week', 'week_of_year']

for col in numeric_cols:
    print(f"\n{col}:")
    print(f"  Min: {calendar_df[col].min()}")
    print(f"  Max: {calendar_df[col].max()}")
    print(f"  Mean: {calendar_df[col].mean():.2f}")
    print(f"  Median: {calendar_df[col].median():.2f}")
    print(f"  Std: {calendar_df[col].std():.2f}")

# Check for unrealistic night values
print("\n" + "=" * 50)
print("Unrealistic Values Check:")
print(f"minimum_nights > 365: {(calendar_df['minimum_nights'] > 365).sum()} rows")
print(f"maximum_nights > 1000: {(calendar_df['maximum_nights'] > 1000).sum()} rows")
print(f"minimum_nights > maximum_nights: {(calendar_df['minimum_nights'] > calendar_df['maximum_nights']).sum()} rows")

# Show distribution of extreme values
if (calendar_df['maximum_nights'] > 1000).sum() > 0:
    print(f"\nExamples of maximum_nights > 1000:")
    print(calendar_df[calendar_df['maximum_nights'] > 1000][['listing_id', 'minimum_nights', 'maximum_nights']].head(10))

In [ ]:
# # CALENDAR DATASET
# ============================================================
# 3: DATA QUALITY CHECKS & VALIDATION
# ============================================================

In [ ]:
# Fix negative maximum_nights values with placeholder
print("Fixing Negative Values:")
print("=" * 50)
print(f"Rows with negative maximum_nights: {(calendar_df['maximum_nights'] < 0).sum()}")

if (calendar_df['maximum_nights'] < 0).sum() > 0:
    print("Replacing negative maximum_nights with 365 (1 year placeholder)...")
    calendar_df.loc[calendar_df['maximum_nights'] < 0, 'maximum_nights'] = 365
    print(f"Fixed! New max value: {calendar_df['maximum_nights'].max()}")

print(f"\nRows with negative minimum_nights: {(calendar_df['minimum_nights'] < 0).sum()}")
if (calendar_df['minimum_nights'] < 0).sum() > 0:
    print("Replacing negative minimum_nights with 1 (minimum stay)...")
    calendar_df.loc[calendar_df['minimum_nights'] < 0, 'minimum_nights'] = 1
    print(f"Fixed!")

# Verify fix
print(f"\nVerification - minimum_nights range: [{calendar_df['minimum_nights'].min()}, {calendar_df['minimum_nights'].max()}]")
print(f"Verification - maximum_nights range: [{calendar_df['maximum_nights'].min()}, {calendar_df['maximum_nights'].max()}]")

In [ ]:
print("DETAILED INVESTIGATION OF PROBLEMS")
print("=" * 80)

# 1. Check negative maximum_nights
negative_max = calendar_df[calendar_df['maximum_nights'] < 0]
print(f"\n1. NEGATIVE MAXIMUM_NIGHTS: {len(negative_max)} rows")
if len(negative_max) > 0:
    print(negative_max[['listing_id', 'date', 'minimum_nights', 'maximum_nights']].head(20))
    print(f"\nUnique listings affected: {negative_max['listing_id'].nunique()}")

# 2. Check extremely high maximum_nights (>2 years)
extreme_max = calendar_df[calendar_df['maximum_nights'] > 730]
print(f"\n2. EXTREMELY HIGH MAXIMUM_NIGHTS (>730 days): {len(extreme_max)} rows")
print(extreme_max['maximum_nights'].describe())
print(f"Unique listings: {extreme_max['listing_id'].nunique()}")

# 3. Check min > max (logical error)
logical_error = calendar_df[calendar_df['minimum_nights'] > calendar_df['maximum_nights']]
print(f"\n3. MIN > MAX (LOGICAL ERROR): {len(logical_error)} rows")
if len(logical_error) > 0:
    print(logical_error[['listing_id', 'minimum_nights', 'maximum_nights']].head(10))

# 4. Check very high minimum_nights
high_min = calendar_df[calendar_df['minimum_nights'] > 365]
print(f"\n4. MINIMUM_NIGHTS > 365 days: {len(high_min)} rows")
if len(high_min) > 0:
    print(high_min[['listing_id', 'minimum_nights', 'maximum_nights']].value_counts().head(10))

In [ ]:
# # CALENDAR DATASET
# ============================================================
# 4: CREATING CLEANED DATA FILE
# ============================================================

In [ ]:
# Export cleaned calendar dataset to CSV (compressed)
calendar_df.to_csv('../data/processed/calendar_cleaned.csv.gz', index=False, compression='gzip')
print("✓ Cleaned calendar dataset exported to: data/processed/calendar_cleaned.csv.gz")
print(f"  Shape: {calendar_df.shape}")
print(f"  Rows: {len(calendar_df):,} | Columns: {len(calendar_df.columns)}")

In [ ]:
# # REVIEWS DATASET
# ============================================================
# # 1: DATA CLEANING & PREPROCESSING
# ============================================================

In [ ]:
#Now we check if there are missing values or duplicates in the reviews dataset
reviews.isna().sum()
reviews.duplicated().sum()

#We just drop the rows with missing values as there are none with duplicates
reviews_clean = reviews.dropna()

#We check again if there are missing values in the cleaned dataset
reviews_clean.isna().sum()

#Now we importying the TfidfVectorizer to transform the text data into numerical data, making it suitable for machine learning algorithms.
from sklearn.feature_extraction.text import TfidfVectorizer

#We create the TF-IDF vectorizer with specific parameters, limiting the number of features to 50 in order to optimise computational time.
vectorizer = TfidfVectorizer(
    stop_words='english', 
    ngram_range=(1, 2),  
    max_features=50)

#We fit and transform the 'comments' column from the cleaned reviews dataset as this is the only text column of the dataset.
texts = reviews_clean['comments'].astype(str)
reviews_final = vectorizer.fit_transform(texts)

#We convert the resulting sparse matrix into a DataFrame for easier analysis and visualization.
tfidf_df = pd.DataFrame(
    reviews_final.toarray(),
    columns=vectorizer.get_feature_names_out())

#We check the first few rows of the TF-IDF DataFrame to see if the tf-idf transformation was successful.
tfidf_df.head()

In [ ]:
# # REVIEWS DATASET
# ============================================================
# 1: CREATING CLEANED DATA FILE
# ============================================================

In [ ]:
# Export cleaned reviews dataset to CSV (compressed)
reviews_clean.to_csv('../data/processed/reviews_cleaned.csv.gz', index=False, compression='gzip')
print("✓ Cleaned reviews dataset exported to: data/processed/reviews_cleaned.csv.gz")
print(f"  Shape: {reviews_clean.shape}")
print(f"  Rows: {len(reviews_clean):,} | Columns: {len(reviews_clean.columns)}")

In [ ]:
# # LISTINGS DATASET
# ============================================================
# 
# ============================================================

In [ ]:
print("=== LISTINGS OVERVIEW ===")
print(f"Shape: {listings.shape}")
print(f"\nColumns: {listings.shape[1]}")
print(listings.columns.tolist())
print(f"\nData types:\n{listings.dtypes}")
print(f"\nMissing values (%):\n{(listings.isnull().sum() / len(listings) * 100).round(2)}")
print(f"\nFirst row:")
listings.head(1)

In [ ]:
# Identify column types for targeted cleaning
print("=== COLUMN ANALYSIS ===\n")

# Numeric columns
numeric_cols = listings.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric columns ({len(numeric_cols)}):\n{numeric_cols}\n")

# Text columns
text_cols = listings.select_dtypes(include=['object']).columns.tolist()
print(f"Text columns ({len(text_cols)}):\n{text_cols}\n")

# Check sample values from text columns
print("=== TEXT COLUMN SAMPLES ===\n")
for col in text_cols[:10]:  # First 10 text columns
    sample = listings[col].dropna().head(2).tolist()
    print(f"{col}: {sample}\n")


In [ ]:
# Identify column types for targeted cleaning
print("=== COLUMN ANALYSIS ===\n")

# Numeric columns
numeric_cols = listings.select_dtypes(include=[np.number]).columns.tolist()
print(f" Numeric columns ({len(numeric_cols)}):")
print(numeric_cols)

# Text columns
text_cols = listings.select_dtypes(include=['object']).columns.tolist()
print(f"\n Text columns ({len(text_cols)}):")
for col in text_cols:
    sample = listings[col].dropna().iloc[0] if len(listings[col].dropna()) > 0 else "N/A"
    print(f"  {col}: {str(sample)[:60]}")

# Boolean columns
print(f"\n Sample values to detect types:")
for col in text_cols[:5]:
    sample = listings[col].dropna().head(2).tolist()
    print(f"  {col}: {sample}")

In [ ]:
# ===== CELL 3: COMPREHENSIVE DATA CLEANING =====

print("=== STARTING DATA CLEANING ===\n")

# 1. CLEAN PERCENTAGE COLUMNS
print("  CLEANING PERCENTAGE COLUMNS")
percentage_cols = [col for col in listings.columns if 'rate' in col.lower() or 'percent' in col.lower()]
print(f"   Found: {percentage_cols}")

for col in percentage_cols:
    if col in listings.columns and listings[col].dtype == 'object':
        listings[col] = listings[col].str.replace('%', '', regex=False).str.strip()
        listings[col] = pd.to_numeric(listings[col], errors='coerce') / 100
        print(f"   ✓ {col} → converted to decimal")

# 2. CLEAN BOOLEAN COLUMNS (t/f to True/False)
print("\n  CLEANING BOOLEAN COLUMNS")
boolean_map = {'t': True, 'f': False, 'T': True, 'F': False}

for col in text_cols:
    if col in listings.columns:
        unique_vals = listings[col].dropna().unique()
        if len(unique_vals) <= 2 and any(v in boolean_map for v in unique_vals):
            listings[col] = listings[col].map(boolean_map)
            listings[col] = listings[col].fillna(False)
            print(f"   ✓ {col} → converted to boolean")

# 3. CLEAN PRICE COLUMNS
print("\n  CLEANING PRICE COLUMNS")
price_cols = [col for col in listings.columns if 'price' in col.lower()]
for col in price_cols:
    if col in listings.columns and listings[col].dtype == 'object':
        listings[col] = listings[col].str.replace('$', '', regex=False)
        listings[col] = listings[col].str.replace(',', '', regex=False)
        listings[col] = pd.to_numeric(listings[col], errors='coerce')
        print(f"   ✓ {col} → cleaned & converted to numeric")

# 4. CLEAN NUMERIC COLUMNS - FILL MISSING
print("\n  FILLING MISSING NUMERIC VALUES")
numeric_cols = listings.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    missing_count = listings[col].isnull().sum()
    if missing_count > 0:
        listings[col].fillna(listings[col].median(), inplace=True)
        print(f"   ✓ {col}: filled {missing_count} missing values with median")

# 5. CLEAN TEXT COLUMNS - FILL MISSING
print("\n  FILLING MISSING TEXT VALUES")
text_cols = listings.select_dtypes(include=['object']).columns.tolist()
for col in text_cols:
    missing_count = listings[col].isnull().sum()
    if missing_count > 0:
        mode_val = listings[col].mode()
        if len(mode_val) > 0:
            listings[col].fillna(mode_val[0], inplace=True)
        else:
            listings[col].fillna('Unknown', inplace=True)
        print(f"   ✓ {col}: filled {missing_count} missing values")

# 6. CONVERT DATE COLUMNS
print("\n  CONVERTING DATE COLUMNS")
date_patterns = ['date', 'since', 'review', 'last']
for col in listings.columns:
    if any(pattern in col.lower() for pattern in date_patterns):
        if listings[col].dtype == 'object':
            listings[col] = pd.to_datetime(listings[col], errors='coerce')
            print(f"   ✓ {col} → converted to datetime")

# 7. DETECT & HANDLE OUTLIERS
print("\n  DETECTING OUTLIERS")
numeric_cols = listings.select_dtypes(include=[np.number]).columns.tolist()
outlier_cols = []

for col in numeric_cols:
    Q1 = listings[col].quantile(0.25)
    Q3 = listings[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ((listings[col] < lower_bound) | (listings[col] > upper_bound)).sum()
    if outliers > 0:
        # CAP instead of remove
        listings[col] = listings[col].clip(lower=lower_bound, upper=upper_bound)
        outlier_cols.append((col, outliers, lower_bound, upper_bound))
        print(f"   ✓ {col}: capped {outliers} outliers [{lower_bound:.2f} - {upper_bound:.2f}]")

# 8. FINAL VALIDATION
print("\n  FINAL VALIDATION")
print(f"   Shape: {listings.shape}")
print(f"   Missing values: {listings.isnull().sum().sum()}")
print(f"   Duplicates: {listings.duplicated().sum()}")

print("\n DATA CLEANING COMPLETE!")

In [ ]:
# # LISTINGS DATASET
# ============================================================
# 4: CREATING CLEANED DATA FILE
# ============================================================

In [ ]:
# Export cleaned listings dataset to CSV (compressed)
listings.to_csv('../data/processed/listings_cleaned.csv.gz', index=False, compression='gzip')
print("✓ Cleaned listings dataset exported to: data/processed/listings_cleaned.csv.gz")
print(f"  Shape: {listings.shape}")
print(f"  Rows: {len(listings):,} | Columns: {len(listings.columns)}")